In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "aac1994"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
# (use the high-res logo provided in Codio; fall back gracefully if not found)
image_filename = 'Grazioso Salvare Logo.png'  # replace with your own image if needed
try:
    encoded_image = base64.b64encode(open(image_filename, 'rb').read())
    logo_img = html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                        style={'height': '64px'})
except Exception:
    logo_img = html.Div()  # no-op if image not available

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.Div([logo_img, html.P('Developer: Mark Christof', style={'margin':'6px 0'})])),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        children=[
            dcc.RadioItems(
                id='filter-type',
                options=[
                    {'label': 'Water Rescue', 'value': 'water'},
                    {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain'},
                    {'label': 'Disaster/Individual Tracking', 'value': 'disaster'},
                    {'label': 'Reset (All)', 'value': 'reset'}
                ],
                value='reset',
                inline=True
            )
        ]

    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
                         page_size=12,
                         sort_action='native',
                         filter_action='native',
                         row_selectable='single',
                         selected_rows=[0],
                         style_table={'overflowX': 'auto'},
                         style_cell={'fontSize': 12, 'padding': '6px'}
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################


# small helper for filter queries that match the project’s rescue profiles
def _filter_query(kind):
    # target dogs <= ~2 years old (104 weeks)
    base = {"animal_type": "Dog", "age_upon_outcome_in_weeks": {"$lte": 104}}
    water = ["Labrador Retriever", "Labrador Retriever Mix", "Chesapeake Bay Retriever",
             "Newfoundland", "Golden Retriever"]
    mountain = ["German Shepherd Dog", "German Shepherd Dog Mix", "Alaskan Malamute",
                "Siberian Husky", "Bernese Mountain Dog", "Australian Shepherd", "Border Collie"]
    disaster = ["German Shepherd Dog", "German Shepherd Dog Mix", "Doberman Pinscher",
                "Rottweiler", "Bloodhound", "Belgian Malinois", "Beagle"]
    if kind == 'water':
        return {"$and": [base, {"breed": {"$in": water}}]}
    if kind == 'mountain':
        return {"$and": [base, {"breed": {"$in": mountain}}]}
    if kind == 'disaster':
        return {"$and": [base, {"breed": {"$in": disaster}}]}
    return {}  # reset


@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    ## FIX ME Add code to filter interactive data table with MongoDB queries
    #
    # Using the CRUD module to read from MongoDB based on the selected filter
    query = _filter_query(filter_type)
    dff = pd.DataFrame.from_records(db.read(query))
    if '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)
    # keep column order stable if possible
    try:
        dff = dff[df.columns]
    except Exception:
        pass
    return dff.to_dict('records')
#        
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#        data=df.to_dict('records')
#       
#       
#        return (data,columns)

# Display the breeds of animal based on quantity represented in the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    try:
        # Accept None/empty safely; fall back to the full df
        dff = pd.DataFrame.from_dict(viewData) if viewData else df.copy()

        # If no data or no 'breed' col, show an empty bar chart instead of erroring
        if dff.empty or 'breed' not in dff.columns:
            fig = px.bar(title='Breed Distribution')
        else:
            top = (
                dff['breed'].astype(str).fillna('Unknown')
                   .replace({'nan': 'Unknown'})
                   .value_counts()
                   .head(10)
                   .reset_index()
                   .rename(columns={'index': 'breed', 'breed': 'count'})
            )
            fig = px.bar(top, x='breed', y='count', title='Top Breeds in Current Selection')

        return [dcc.Graph(figure=fig)]
    except Exception as e:
        # Fail soft: print to server log and render a friendly placeholder
        print("update_graphs error:", e)
        return [html.Div("Chart unavailable for this selection.")]

    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in (selected_columns or [])]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server(mode='jupyterlab', host='0.0.0.0', port=8050)



update_graphs error: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['count', 'count'] but received: breed
update_graphs error: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['count', 'count'] but received: breed
update_graphs error: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['count', 'count'] but received: breed
update_graphs error: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['count', 'count'] but received: breed
update_graphs error: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['count', 'count'] but received: breed
update_graphs error: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['count', 'count'] but received: breed
